# Pré seleção - Implantação

Remove atributos de acordo com os seguintes critérios: 
- Variabilidade próximo de 0
- Alta correlação entre si
- Lida com NaN e valores faltantes

Este notebook apresenta:
- como utilizar o [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como utilizar um modelo para fornecer predições em tempo real.

## Declaração de Transformador Personalizado

In [ ]:
%%writefile CustomTransformer.py
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class Correlation(BaseEstimator, TransformerMixin):
    """Feature selector that removes correlated features.
    
    This feature selection algorithm looks only at features(X) and 
    and removes those with high correlation.
    
    Attributes:
        categorical_indexes: a np.ndarray of categoricals indexes.
        cutoff: float of cutoff.
        drop_indexes: list of indexes to be droped.
    """

    def __init__(self, categorical_indexes: np.ndarray, cutoff: float):
        """Inits Correlation class.
        
        Args:
            categorical_indexes: categorical indexes of X.
            cutoff: cutoff value.
        """
        self.categorical_indexes = categorical_indexes
        self.cutoff = cutoff
    
    def transform(self, X) -> np.ndarray:
        """Reduce X to the selected features.
        
        Args:
            X: the input samples.
            
        Returns:
            np.ndarray: the input samples without only the selected features.
        """
        return np.delete(X, np.unique(self.drop_indexes), axis=1)
    
    def get_support(self) -> np.ndarray:
        """Returns a list of indexes to be removed.
        
        Returns:
            np.ndarray: indexes removed by the model.
        """
        return np.unique(self.drop_indexes)
    
    def fit(self, X: np.ndarray, y=None) -> np.ndarray:
        """Fit the model.
        
        Learn correlated features from X.
        
        Args:
            X: the imput sample.
        
        Returns:
            self
        """
        # get only numerical values from X
        X_numerical = np.delete(X, self.categorical_indexes, axis=1)
        
        # check the shape of input
        if np.ma.size(X_numerical, axis=0) <= 1 \
        or np.ma.size(X_numerical, axis=1) <= 1:
            return X
        
        # correlation matrix
        corr_matrix = np.abs(np.corrcoef(X_numerical.astype(float), rowvar=False))
        
        # mean correlation for each column
        mean_corr = np.mean(corr_matrix, axis=1)

        # pairwise correlations above cutoff
        above_cutoff = np.argwhere(np.triu(corr_matrix, k=1) > self.cutoff)

        # for each pairwise correlation above cutoff
        # remove the feature with the highest mean correlation 
        self.drop_indexes = [
            above_cutoff[i, np.argmax(pair)] 
            for i, pair in enumerate(mean_corr[above_cutoff])]

        return self

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real. <br>
Para isso você deve criar uma classe Model que implementa o método predict.

In [ ]:
%%writefile Model.py
import logging
from typing import List, Iterable, Dict, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Model(object):    
    def __init__(self, dataset: str = None, target: str = None):
        # Carrega artefatos: estimador, etc
        model = load_model()
        self.pipeline = model["pipeline"]
        self.features_names_training = model["columns"]
        self.features_after_pipeline = model["features_after_pipeline"]

    def class_names(self):
        return self.features_after_pipeline.tolist()

    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:
        # Antes de alimentar o modelo com X, reordena seus atributos com os dados de treinamento
        if feature_names:
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.features_names_training].to_numpy()

        # Realiza transformação
        X = self.pipeline.transform(X)

        return X

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "SepalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [4.3, 7.9]
        },
        {
            "name": "SepalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [2.0, 4.4]
        },
        {
            "name": "PetalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 6.9]
        },
        {
            "name": "PetalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.1, 2.5]
        }
    ],
    "targets": [
        {
            "name": "SepalLengthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [4.3, 7.9]
        },
        {
            "name": "SepalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [2.0, 4.4]
        },
        {
            "name": "PetalWidthCm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [0.1, 2.5]
        }
    ]
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")